In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from datasets import Dataset

data = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/IMDB-Dataset.csv')

#data.head()

dataset = Dataset.from_pandas(data)

dataset = dataset.train_test_split(test_size=0.3)

In [4]:
label2id = {'negative': 0, 'positive': 1}
id2label = {0:'negative', 1:'positive'}

dataset = dataset.map(lambda x: {'label': label2id[x['sentiment']]})

Map: 100%|██████████| 15000/15000 [00:00<00:00, 17247.20 examples/s]


In [6]:
dataset['train'][0]

dataset['train'][0].keys()

dict_keys(['review', 'sentiment', 'label'])

In [8]:
from transformers import AutoTokenizer
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_ckpt = 'huawei-noah/TinyBERT_General_4L_312D'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

tokenizer(dataset['train'][0]['review'])

def tokenize(batch):
    temp = tokenizer(batch['review'], padding=True, truncation=True, max_length=300)
    return temp

dataset = dataset.map(tokenize, batched=True, batch_size=None)

Map: 100%|██████████| 15000/15000 [00:04<00:00, 3660.78 examples/s]


In [11]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=len(label2id), label2id=label2id, id2label=id2label)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
args = TrainingArguments(
    output_dir='train_dir',
    overwrite_output_dir=True,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [14]:
trainer.train()

 15%|█▌        | 500/3282 [19:45<1:53:31,  2.45s/it]

{'loss': 0.4625, 'grad_norm': 8.376426696777344, 'learning_rate': 1.695307739183425e-05, 'epoch': 0.46}


 30%|███       | 1000/3282 [38:41<1:26:53,  2.28s/it]

{'loss': 0.3523, 'grad_norm': 7.931914329528809, 'learning_rate': 1.3906154783668494e-05, 'epoch': 0.91}


                                                     
 33%|███▎      | 1094/3282 [50:31<1:12:35,  1.99s/it]

...

 91%|█████████▏| 3000/3282 [2:13:19<11:25,  2.43s/it]

{'loss': 0.2558, 'grad_norm': 15.98099136352539, 'learning_rate': 1.7184643510054846e-06, 'epoch': 2.74}


                                                     
100%|██████████| 3282/3282 [2:34:02<00:00,  2.82s/it]

{'eval_loss': 0.28493577241897583, 'eval_accuracy': 0.8814666666666666, 'eval_runtime': 531.4217, 'eval_samples_per_second': 28.226, 'eval_steps_per_second': 0.883, 'epoch': 3.0}
{'train_runtime': 9242.0322, 'train_samples_per_second': 11.361, 'train_steps_per_second': 0.355, 'train_loss': 0.3178028662273028, 'epoch': 3.0}


TrainOutput(global_step=3282, training_loss=0.3178028662273028, metrics={'train_runtime': 9242.0322, 'train_samples_per_second': 11.361, 'train_steps_per_second': 0.355, 'total_flos': 882184338000000.0, 'train_loss': 0.3178028662273028, 'epoch': 3.0})

In [15]:
trainer.evaluate()

100%|██████████| 469/469 [09:29<00:00,  1.21s/it]


{'eval_loss': 0.28493577241897583,
 'eval_accuracy': 0.8814666666666666,
 'eval_runtime': 570.7928,
 'eval_samples_per_second': 26.279,
 'eval_steps_per_second': 0.822,
 'epoch': 3.0}

In [16]:
trainer.save_model('tinybert-sentiment-analysis')

In [17]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the movie is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

from transformers import pipeline

import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

classifier = pipeline('text-classification', model='tinybert-sentiment-analysis', device=device)

classifier(data)

[{'label': 'negative', 'score': 0.9897700548171997},
 {'label': 'negative', 'score': 0.9893438816070557},
 {'label': 'positive', 'score': 0.9897921085357666}]